# Convergence Studies & Equation of State

Learn to run convergence studies and EOS (Equation of State) workflows using the CrystalMath Python API.

**Prerequisites:**
- Run `uv sync` in the crystalmath repository root
- This notebook works with real workflow APIs (no stubs!)

**What you'll learn:**
1. K-point/SHRINK convergence studies
2. Energy cutoff convergence (VASP)
3. Generating input files for each parameter value
4. Analyzing convergence results
5. Running EOS calculations for bulk modulus
6. Fitting Birch-Murnaghan equation of state

## 1. Imports

In [ ]:
from crystalmath.workflows.convergence import (
    ConvergenceStudy,
    ConvergenceStudyConfig,
    ConvergenceParameter,
    ConvergencePoint,
    ConvergenceStudyResult,
)
from crystalmath.workflows.eos import (
    EOSWorkflow,
    EOSConfig,
    EOSPoint,
    EOSResult,
)
from crystalmath.api import CrystalController
from crystalmath.models import JobSubmission, DftCode

print("Imports successful!")


## 2. K-Point Convergence Study

The most common convergence test is k-point mesh density. For CRYSTAL, this is controlled by the `SHRINK` parameter.

In [ ]:
# Define a base CRYSTAL input
base_input = """MgO SCF
CRYSTAL
0 0 0
225
4.21
2
12  0.0  0.0  0.0
8   0.5  0.5  0.5
END
12 3
8 3
END
SHRINK
8 8
TOLDEE
8
END
END
"""

# Configure convergence study
config = ConvergenceStudyConfig(
    parameter=ConvergenceParameter.SHRINK,
    values=[4, 6, 8, 10, 12, 14, 16],
    base_input=base_input,
    energy_threshold=0.001,  # eV/atom convergence criterion
    dft_code="crystal",
    name_prefix="mgo_conv",
)

study = ConvergenceStudy(config)
print(f"Convergence parameter: {config.parameter.value}")
print(f"Values to test: {config.values}")
print(f"Threshold: {config.energy_threshold} eV/atom")


## 3. Generate Input Files

The `generate_inputs()` method creates modified input files for each parameter value.

In [ ]:
# Generate (name, input_content) pairs for each parameter value
inputs = study.generate_inputs()

print(f"Generated {len(inputs)} input files:")
for name, content in inputs:
    # Show the job name for each SHRINK value
    print(f"  {name}")


In [ ]:
# Inspect one of the generated inputs
name, content = inputs[2]  # SHRINK=8
print(f"=== {name} ===")
print(content)


## 4. Submit Convergence Jobs

Submit each convergence point using the `CrystalController` API.

In [ ]:
ctrl = CrystalController(db_path="convergence_demo.db")

# Submit each convergence point
pks = []
for name, content in inputs:
    job = JobSubmission(
        name=name,
        dft_code=DftCode.CRYSTAL,
        input_content=content,
    )
    pk = ctrl.submit_job(job)
    pks.append(pk)
    print(f"  Submitted {name} -> PK={pk}")


## 5. Analyze Results

After jobs complete, analyze the convergence using the `analyze_results()` method.

In [ ]:
# In a real workflow, you would extract energies from completed jobs:
# energies = [ctrl.get_job_details(pk).final_energy for pk in pks]

# For demonstration, simulate converged energies (in eV/atom)
demo_energies = [
    -275.123,  # SHRINK=4
    -275.456,  # SHRINK=6
    -275.489,  # SHRINK=8
    -275.493,  # SHRINK=10
    -275.494,  # SHRINK=12
    -275.494,  # SHRINK=14
    -275.494,  # SHRINK=16
]

# Update each point with the computed energy
for i, energy in enumerate(demo_energies):
    study.update_point(
        i,
        energy_per_atom=energy,
        status="completed",
    )

# Analyze convergence
result = study.analyze_results()

print(f"Convergence Analysis:")
print(f"  Converged: {result.converged_value is not None}")
print(f"  Converged value: SHRINK = {result.converged_value}")
print(f"  Converged at index: {result.converged_at_index}")
print(f"  Recommendation: {result.recommendation}")


In [ ]:
# Result as dictionary (for serialization or storage)
result_dict = result.to_dict()
print("Result dict keys:", list(result_dict.keys()))
print("\nFirst convergence point:")
print(result_dict['points'][0])


## 6. Other Convergence Parameters

The convergence framework supports multiple parameter types for different DFT codes.

In [ ]:
# Available convergence parameters
print("Available convergence parameters:")
for param in ConvergenceParameter:
    print(f"  {param.value}")

# Example: ENCUT convergence for VASP
vasp_config = ConvergenceStudyConfig(
    parameter=ConvergenceParameter.ENCUT,
    values=[300, 350, 400, 450, 500, 550, 600],
    base_input="VASP INCAR content...",
    energy_threshold=0.001,
    dft_code="vasp",
    name_prefix="encut_conv",
)

print(f"\nVASP ENCUT convergence: {vasp_config.values}")


## 7. Equation of State (EOS)

Calculate bulk modulus from energy-volume data using the Birch-Murnaghan equation of state.

In [ ]:
# Configure EOS workflow
eos_config = EOSConfig(
    source_job_pk=1,  # PK of a completed SCF job with optimized geometry
    volume_range=(0.90, 1.10),  # -10% to +10% volume scaling
    num_points=7,
    eos_type="birch_murnaghan",
    dft_code="crystal",
    name_prefix="mgo_eos",
)

eos = EOSWorkflow(eos_config)
print(f"EOS Configuration:")
print(f"  Volume range: {eos_config.volume_range}")
print(f"  Number of points: {eos_config.num_points}")
print(f"  EOS type: {eos_config.eos_type}")


## 8. Generate Volume Points

Generate structures at different volumes by isotropic scaling of the cell.

In [ ]:
# Generate strained structures
# In practice, cell/positions/symbols come from the source job
demo_cell = [[4.21, 0, 0], [0, 4.21, 0], [0, 0, 4.21]]
demo_positions = [[0, 0, 0], [0.5, 0.5, 0.5]]
demo_symbols = ["Mg", "O"]

points = eos.generate_volume_points(demo_cell, demo_positions, demo_symbols)

print(f"Generated {len(points)} volume points:")
for p in points:
    print(f"  Scale: {p['volume_scale']:.3f}, Volume: {p['volume']:.2f} A^3")


## 9. Fit EOS

After running all volume points and collecting energies, fit the equation of state.

In [ ]:
# In a real workflow, you would:
# 1. Submit SCF jobs for each volume point
# 2. Collect energies: eos.update_point(i, energy=e, status="completed")
# 3. Fit EOS: result = eos.fit_eos()

# Example result structure (simulated)
result = EOSResult(
    status="completed",
    v0=74.5,    # Equilibrium volume (A^3)
    e0=-275.5,  # Equilibrium energy (eV)
    b0=160.0,   # Bulk modulus (GPa)
    bp=4.2,     # Pressure derivative B'
    eos_type="birch_murnaghan",
)

print(f"EOS Fit Results:")
print(f"  V0 = {result.v0} A^3")
print(f"  E0 = {result.e0} eV")
print(f"  B0 = {result.b0} GPa")
print(f"  B' = {result.bp}")
print(f"  Type: {result.eos_type}")
print(f"\nAs dict: {result.to_dict()}")


## Summary

In this notebook, you learned how to:

1. **Set up convergence studies** with `ConvergenceStudyConfig`
2. **Generate input files** for each parameter value
3. **Submit jobs** using `CrystalController`
4. **Analyze convergence** and get recommendations
5. **Configure EOS workflows** for bulk modulus calculations
6. **Generate volume-scaled structures** for EOS fitting
7. **Fit Birch-Murnaghan EOS** to extract elastic properties

**Next Steps:**
- Try `05_templates_workflows.ipynb` to learn about the template system and phonon workflows
- Explore advanced workflows like band structure and phonon calculations
- Combine convergence studies with production calculations